In [ ]:


import os
import pandas as pd

from templateMaker.content_population import generate_number, generate_student_name, replace_placeholders_in_word, split_and_populate_explanation
from templateMaker.data_preparation import load_word_template

def sanitize_folder_name(name):
    """Removes invalid characters from folder names."""
    return "".join(char for char in name if char.isalnum() or char in " _-")

def create_folder_structure(df):
    """Creates the folder structure, populates templates, and exports files."""
    for main_class in df["التصنيف الرئيسي مدقق"].unique():
        sub_df = df[df["التصنيف الرئيسي مدقق"] == main_class]
        for sub_class_list in sub_df["التصنيفات الفرعية مدققة"].str.split(r"[،,]"):
            for sub_class in sub_class_list:
                filtered_df = sub_df[
                    sub_df["التصنيفات الفرعية مدققة"].str.contains(sub_class.strip())
                ]

                for index, row in filtered_df.iterrows():
                    model_number = index - filtered_df.index[0] + 1
                    # --- Content Population ---
                    word_template = load_word_template()
                    if word_template:
                        data = row.to_dict()
                        data["Model Number"] = model_number
                        data["رقم"] = generate_number(data)
                        data["اسم الطالب"] = generate_student_name(data)
                        print(data)
                        # replace_placeholders_in_word(word_template, data)

                        # if len(data["الشرح مدقق"]) > 500:
                        #     word_template = split_and_populate_explanation(
                        #         word_template, data["الشرح مدقق"]
                        #     )

                        # word_file_path = os.path.join(
                        #     model_folder, f"نموذج {model_number}.docx"
                        # )
                        # word_template.save(word_file_path)

                    # PowerPoint Population (similar logic)
                    # powerpoint_template = load_powerpoint_template()
                    # if powerpoint_template:
                    #     data = row.to_dict()
                    #     data["Model Number"] = model_number
                    #     data["رقم"] = generate_number(data)
                    #     data["اسم الطالب"] = generate_student_name(data)

                    #     replace_placeholders_in_powerpoint(powerpoint_template, data)

                    #     # if row["اسم الصورة"]:
                    #     #     image_path = os.path.join(IMAGE_FOLDER, row["اسم الصورة"])
                    #     #     slide = powerpoint_template.slides[0]
                    #     #     # left = PtInches(1)
                    #     #     # top = PtInches(2)
                    #     #     # slide.shapes.add_picture(image_path, left, top)
                    #     #     slide.shapes.add_picture(image_path, 0, 0)

                    #     pptx_file_path = os.path.join(
                    #         model_folder, f"نموذج {model_number}.pptx"
                    #     )
                    #     powerpoint_template.save(pptx_file_path)

                    # # --- File Exports ---
                    # if word_template:

                    #     pdf_file_path = os.path.join(
                    #         model_folder, f"نموذج {model_number}.pdf"
                    #     )
                    #     convert_docx_to_pdf(word_file_path, pdf_file_path)

                    # if powerpoint_template:
                    #     mp4_file_path = os.path.join(
                    #         model_folder, f"نموذج {model_number}.mp4"
                    #     )
                    #     convert_pptx_to_mp4(pptx_file_path, mp4_file_path)


def template_maker():
    """Main function to execute the entire process."""
    df = read_excel_data()
    create_folder_structure(df)

template_maker()

In [7]:
from datetime import datetime
import itertools
import os
import random
import shutil
import subprocess
import time
import cv2
import pandas as pd
from docx import Document
from pptx import Presentation
from pptx.util import Inches, Pt
from docx.shared import Pt as DocxPt
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docxtpl import DocxTemplate
from moviepy.editor import *
import comtypes.client
from docx2pdf import convert
import aspose.slides as slides
from pptx import Presentation
from pptx.enum.shapes import MSO_SHAPE_TYPE
import os
from PIL import Image
from pdf2image import convert_from_path
from pptxtopdf import convert as converttopdf
# Constants
EXCEL_FILE = 'Bashar - Copy.xlsx'
WORD_TEMPLATE = 'templateMaker/Main.docx'
PPT_TEMPLATE = 'templateMaker/Main.pptx'
OUTPUT_DIR = 'output'


def generate_number():
  """Generate a 9-digit number."""
  digits = random.sample(range(10), 9)  # Ensure unique digits
  return "".join(map(str, digits))

def find_expression(number):
  """Find an expression that evaluates to 100."""
  digits = list(number)
  operators = ['+', '-', '*']
  for perm in itertools.permutations(digits): # Try all digit permutations
    for ops in itertools.product(operators, repeat=len(digits) - 1): # Try all operator combinations
      expression = ""
      for i, digit in enumerate(perm):
        expression += digit
        if i < len(perm) - 1:
          expression += ops[i]

      try:
        if eval(expression) == 100:
          return expression + " = 100"
      except (ZeroDivisionError, SyntaxError): 
        pass  # Handle potential errors

  return "No solution found" # If no solution is found


# PLACEHOLDERS = {
#     "StudentName": "اسم الطالب",
#     "نص السؤال مدقق": "نص السؤال مدقق",
#     "الخيار أ مدقق": "الخيار أ مدقق",
#     "الخيار ب مدقق": "الخيار ب مدقق",
#     "الخيار ج مدقق": "الخيار ج مدقق",
#     "الخيار د مدقق": "الخيار د مدقق",
#     # ... other placeholders
# }

def populate_word_template(template_path, data, output_path):
    """Populate Word template with data."""
    # doc = Document(template_path)
    # for paragraph in doc.inline_shapes:
    #     print(paragraph.type)
    #     for run in paragraph.runs:  # Loop through runs
    #         print(run.text)
    #         for key, value in PLACEHOLDERS.items():
    #             print(run.text)
    #             placeholder = f"[{value}]"
    #             print("placeholder: " + placeholder)
    #             if placeholder in run.text:
    #                 run.text = run.text.replace(placeholder, str(data[key]))

    # Add math expression
    # math_para = doc.add_paragraph()
    # math_para.alignment = WD_ALIGN_PARAGRAPH.CENTER
    # math_run = math_para.add_run(generate_math_expression(generate_number()))
    # math_run.font.size = DocxPt(14)
    # math_run.font.bold = True
    
    
    doc = DocxTemplate(template_path)
    doc.render(data)
    doc.save(output_path)

    # doc.save(output_path)

def populate_ppt_template(template_path, data, output_path):
    """Populate PowerPoint template with data."""
    # prs = Presentation(template_path)
    
    prs = Presentation(template_path)

    # Iterate through slides and replace placeholders
    for slide in prs.slides:
        for shape in slide.shapes:
            if shape.has_text_frame:
                for paragraph in shape.text_frame.paragraphs:
                    for run in paragraph.runs:
                        # Replace placeholders with actual content
                        for key, value in data.items():
                            if key in run.text:
                                run.text = run.text.replace(key, str(value))

    # Save the modified presentation
    prs.save(output_path)
    
def convert_pptx_to_mp4_ffmpeg(pptx_path, mp4_path, duration=5, fps=30):
    """
    Convert PowerPoint to MP4 using ffmpeg.
    :param pptx_path: Path to the PowerPoint file
    :param mp4_path: Path to save the MP4 file
    :param duration: Duration (in seconds) for each slide
    :param ffmpeg_path: Path to ffmpeg executable
    """
    # Extract images from PowerPoint
    prs = Presentation(pptx_path)
    img_dir = os.path.splitext(pptx_path)[0] + '_images'
    os.makedirs(img_dir, exist_ok=True)

    for i, slide in enumerate(prs.slides):
        img_path = os.path.join(img_dir, f'slide_{i+1}.png')
        img = slide.shapes.title.text  # This is a placeholder, we need to render the slide
        # For now, we'll create a blank image with text
        img = Image.new('RGB', (1280, 720), color='white')
        img.save(img_path)

    # Create video from images
    images = [img for img in os.listdir(img_dir) if img.endswith(".png")]
    images.sort(key=lambda x: int(x.split('_')[1].split('.')[0]))

    frame = cv2.imread(os.path.join(img_dir, images[0]))
    height, width, layers = frame.shape

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter(mp4_path, fourcc, fps, (width, height))

    for image in images:
        frame = cv2.imread(os.path.join(img_dir, image))
        for _ in range(duration * fps):
            video.write(frame)

    cv2.destroyAllWindows()
    video.release()

    # Clean up the temporary image directory
    for file in os.listdir(img_dir):
        os.remove(os.path.join(img_dir, file))
    os.rmdir(img_dir)
    
def convert_ppt_to_mp4(ppt_input, mp4_output):
    """Converts a PowerPoint presentation to an MP4 video."""

    prs = Presentation(ppt_input)

    # Create a list of image files from the slides
    image_files = []
    for i, slide in enumerate(prs.slides):
        slide_image_path = os.path.join(os.path.dirname(ppt_input), f"slide_{i}.png")
        slide.shapes[0].save_as_png(slide_image_path)  # Correct way to save slide as PNG
        image_files.append(slide_image_path)

    # Create a video clip from the images
    clips = [ImageClip(img_file).set_duration(5) for img_file in image_files]  # Adjust duration as needed
    video = concatenate_videoclips(clips, method="compose")

    # Export the video to MP4
    video.write_videofile(mp4_output, fps=24)

    # Remove the temporary image files
    for img_file in image_files:
        os.remove(img_file)

def ppt_to_mp4(ppt_path, mp4_path):
    # Use a library like "moviepy" or "python-pptx" to convert
    # Alternatively, you can use external tools like ffmpeg
    try:
        subprocess.run(["ffmpeg", "-i", ppt_path, mp4_path])
        print(f"Successfully converted {ppt_path} to {mp4_path}")
        
        # clip = VideoFileClip(ppt_path)  # Your ppt file path
        # clip.write_videofile(mp4_path)
    except FileNotFoundError:
        print("FFmpeg not found! Please install it to convert PPT to MP4.")
    
    # with slides.Presentation(ppt_path) as presentation:
        # options = slides.export.GifOptions()
        # options.default_delay = 3000  # Set the delay between slides (in milliseconds)
        # options.transition_fps = 35  # Set the frame rate for slide transitions
        # # You can adjust other options as needed

        # # Export the presentation to an MP4 video
        # presentation.save(mp4_path, slides.export.SaveFormat.MP4)
        # fps = 1
        # with slides.export.PresentationEnumerableFramesGenerator(presentation, fps) as frames_stream:
        #     for frame_args in frames_stream.enumerate_frames(presentation.slides):
        #         frame_args.get_frame().save(f"frame_{frame_args.frames_generator.frame_index:04d}.png")
    
    # clip = VideoFileClip(ppt_path)
    # clip.write_videofile(mp4_path, fps=24)
    # print(f"Successfully converted {ppt_path} to {mp4_path}")

def read_excel_data(file_path=EXCEL_FILE):
    """Reads data from the Excel file."""
    df = pd.read_excel(file_path)
    return df


def convert_pdf_to_images(pdf_file_path, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    images = convert_from_path(pdf_file_path)
    
    image_paths = []
    for i, img in enumerate(images):
        image_path = os.path.join(output_dir, f"slide_{i + 1}.png")
        img.save(image_path, 'PNG')
        image_paths.append(image_path)
    
    return image_paths

def create_video_from_images(image_dir, video_output_path, frame_rate=1):
    images_pattern = os.path.join(image_dir, "slide_%d.png")
    cmd = [
        "ffmpeg", "-framerate", str(frame_rate), "-i", images_pattern,
        "-c:v", "libx264", "-r", "30", "-pix_fmt", "yuv420p", video_output_path
    ]
    subprocess.run(cmd)
    
def convert_ppt_to_pdf(ppt_file_path, pdf_output_path):
    converttopdf(ppt_file_path, pdf_output_path)
    
def main():
    # shutil.rmtree(OUTPUT_DIR)
    # Read Excel data
    df = read_excel_data(EXCEL_FILE)
    # Create output directory if it doesn't exist
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    # Process each row in the Excel file
    
    for index, row in df[:1].iterrows():
        # Create subdirectories based on categories
        main_category = row['التصنيف الرئيسي مدقق']
        sub_categories = row['التصنيفات الفرعية مدققة'].split(',')
        for sub_category in sub_categories:
            sub_dir = os.path.join(OUTPUT_DIR, main_category, sub_category.strip())
            os.makedirs(sub_dir, exist_ok=True)

            # Generate file names
            word_output = os.path.join(sub_dir, f'نموذج_{index+1}.docx')
            pdf_file_path = os.path.join(
                            sub_dir, f"نموذج {index+1}.pdf"
                        )
            ppt_output = os.path.join(sub_dir, f'نموذج_{index+1}.pptx')
            mp4_output = os.path.join(sub_dir, f'نموذج_{index+1}.mp4')
            print(ppt_output)
            print(mp4_output)
            number = generate_number()
            expression_number = find_expression(number)
            PLACEHOLDERS = {
                "studentName":"someone",
                "number": number,
                "expressionNumber":expression_number,
                "modelNumber": index+1,
                "date": datetime.today().strftime('%Y-%m-%d %H:%M:%S'),
                "questionsNumber":"31",
                "studentsResults":"100%",
                "question": row['نص السؤال مدقق'],
                # "الخيار_أ_مدقق": row['الخيار أ مدقق'],
                # "الخيار_ب_مدقق": row['الخيار ب مدقق'],
                # "الخيار ج مدقق": row['الخيار ج مدقق'],
                # "الخيار د مدقق": row['الخيار د مدقق'],
            }
            # Populate templates
            # populate_word_template(WORD_TEMPLATE, PLACEHOLDERS, word_output)
            # convert(word_output, pdf_file_path)
            # print(f"Successfully converted {word_output} to {pdf_file_path}")

            populate_ppt_template(PPT_TEMPLATE, PLACEHOLDERS, ppt_output)
            
            # Convert PPT to MP4
            # convert_ppt_to_mp4(ppt_output, mp4_output)
            
            # ppt_to_mp4(ppt_output, mp4_output)
            
            # ppt_file_path = ppt_output  # Path to your PowerPoint file
            # image_dir = "slides"

            # pdf_output_path = ppt_file_path.replace('.pptx', '.pdf')
            # convert_ppt_to_pdf(ppt_file_path, pdf_output_path)
            # # Step 1: Convert PPT to Images
            # image_paths = convert_pdf_to_images(pdf_output_path, image_dir)
            
            # # Step 2: Convert Images to Video
            # create_video_from_images(image_dir, mp4_output)

    print("Processing complete. Check the 'output' directory for results.")

if __name__ == "__main__":
    main()

output\الأعداد الطبيعية والنسبية\المتوسط الحسابي، الوسيط، المنوال\نموذج_1.pptx
output\الأعداد الطبيعية والنسبية\المتوسط الحسابي، الوسيط، المنوال\نموذج_1.mp4


100%|██████████| 1/1 [00:04<00:00,  4.03s/it]


 
0512345678
ألف بن باء آل جيم
رقم النموذج: 
modelNumber
date
تنتهي  المقدمة في الدقيقة 0:30
أنت وثقت فينا وعطيتنا وقتك، واحنا نشكر لك هذي الثقة، ورح نتأكد أننا ما نخيب ظنك أبدا.
رح نظل معك طول الطريق، وما رح نقصر معك بأي شيء نقدر نخدمك فيه، ونوعدك أننا رح نبذل كل جهدنا حتى نوصلك للدرجة الكاملة في الجانب الكمي، ونفرح مع بعض بتألقك ونجاحك :)
لا تظن أن الـ 100% بعيدة عليك، بالعكس، ترى كل شيء يأكد لنا أنك تقدر، وحتى رقم جوالك اكتشفنا أنك تقدر ترتبه حتى يعطيك الرقم 100 
😅
نجم المستقبل ألف بن باء آل جيم
تنتهي  المقدمة في الدقيقة 0:30
نجم المستقبل 
studentName
number
expressionNumber
الـ 100 تكون واضحة لأهلها
وأنت منهم بإذن الله، اطمئن :)
تنتهي  المقدمة في الدقيقة 0:30
طلبت الاختبار، واحنا بأمرك. سوينا اختبار مخصص وفريد لك. وأيضا ربطنا الاختبار بنموذج ذكاء اصطناعي حتى يدرس نتائجك ويعرف نقاط ضعفك وقوتك، ويفهم عقليتك وطريقة تفكيرك بشكل أفضل، وبالتالي يرتب لك أفضل وأسرع خطة تدريب حتى توصل للعلامة الكاملة -بإذن الله-.
وقِّت لنفسك 25 دقيقة لأخذ الاختبار، وبعدها اِبدأ، وبعد ما تخلص بلِّغنا بأجوبتك

In [10]:
import itertools
import random

def generate_number():
  """Generate a 9-digit number."""
  digits = random.sample(range(10), 9)  # Ensure unique digits
  return "".join(map(str, digits))

def find_expression(number):
  """Find an expression that evaluates to 100."""
  digits = list(number)
  operators = ['+', '-', '*']
  for perm in itertools.permutations(digits): # Try all digit permutations
    for ops in itertools.product(operators, repeat=len(digits) - 1): # Try all operator combinations
      expression = ""
      for i, digit in enumerate(perm):
        expression += digit
        if i < len(perm) - 1:
          expression += ops[i]

      try:
        if eval(expression) == 100:
          return expression + " = 100"
      except (ZeroDivisionError, SyntaxError): 
        pass  # Handle potential errors

  return "No solution found" # If no solution is found


number = generate_number()
print(number)
print(find_expression(number))

710248695
7+1*0+2+4*8+6*9+5 = 100


In [1]:
import os

# Define the output file name
output_file = "combined_output.py"

# Open the output file in write mode
with open(output_file, 'w') as outfile:
    # Walk through the current directory and subdirectories
    for root, dirs, files in os.walk('.'):
        for file in files:
            # Check if the file is a Python file
            if file.endswith('.py') and file != output_file:
                # Get the full path of the Python file
                file_path = os.path.join(root, file)
                
                # Write a header with the file path
                outfile.write(f"# File: {file_path}\n\n")
                
                # Open and read the file
                with open(file_path, 'r') as infile:
                    # Write the content of the file to the output file
                    outfile.write(infile.read() + "\n\n")
                    
print(f"All Python files have been combined into {output_file}")


All Python files have been combined into combined_output.py


In [3]:
import os

def combine_python_files(directory):
  """Combines all .py and .python files in a directory and its subdirectories into a single file.

  Adds a comment above each inserted code block indicating the source file path.

  Args:
    directory: The directory to search for Python files.
  """

  output_file = "combined_code_2.py"
  with open(output_file, "w") as outfile:
    for root, _, files in os.walk(directory):
      for file in files:
        if file.endswith((".py", ".python")):
          filepath = os.path.join(root, file)
          relative_path = os.path.relpath(filepath, directory)

          outfile.write(f"# --- Start of code from: {relative_path} ---\n")
          with open(filepath, "r") as infile:
            outfile.write(infile.read())
          outfile.write(f"# --- End of code from: {relative_path} ---\n\n")

  print(f"Combined Python files into: {output_file}")

if __name__ == "__main__":
  current_directory = os.getcwd()  # Get the current directory
  combine_python_files(current_directory)

Combined Python files into: combined_code_2.py
